In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.special import digamma
from scipy.stats import dirichlet
import time
from scipy.special import gamma as Gamma

### Latent Dirichlet Allocation (LDA). 
Please download the pre-processed corpus from `data/lda`. From `ap.txt`, you can see the original Associated Press corpus. `ap.dat` are the processed data which you will work on. Each row in `ap.dat` represents a document. In total, we have $2,246$ documents. The first number in each row is the number of words in the document. The following are a list of  **word-id:count** items. Word-id starts with 0. The corresponding word list is given in `vocab.txt`. The first row corresponds to Word-id 0, second, Word-id 1, and continue. 
The first number in each row is the number of words in the document. The following are a list of word-id:count items. Word-id starts with 0. The corresponding word list is given in `vocab.txt`. The first row correspond to Word-id 0, second, Word-id 1, and continue.

In [2]:
def softmax(x, axis = 1):
    z = x - np.max(x, axis = 1, keepdims = True)
    numerator = np.exp(z)
    denominator = np.sum(numerator, axis = 1, keepdims = True)
    softmax = numerator/denominator
    return softmax

### Data preparation!

In [3]:
h = open("data/lda/vocab.txt", "r")
Vocab = h.read().splitlines()

In [4]:
Vocab[:5]

['i', 'new', 'percent', 'people', 'year']

In [5]:
V = len(Vocab)
V # number of vocabulary

10473

In [6]:
f = open("data/lda/ap.dat", "r")
list_Documents = f.read().splitlines()

In [7]:
M = len(list_Documents)
M # number of documnets

2246

In [8]:
def len_of_documents():
    l_o_d = []
    for i in range(M):
        l_o_d.append(int(list_Documents[i].split(' ')[0]))
    return l_o_d
lengths_of_documents = len_of_documents()

In [9]:
def start_position_documents():
    L = [0]
    for x in lengths_of_documents:
        L.append(x+L[-1])
    return L
spd = start_position_documents()

In [10]:
spd[-1]

302031

In [11]:
len(spd)

2247

In [12]:
len(lengths_of_documents)

2246

In [13]:
def _W_():
    W = []
    for m in range(M):
        c = 0
        l = lengths_of_documents[m]
        W.append(np.zeros(l).astype(int))
        L = list_Documents[m].split(' ')[1:]

        for w_n in L:
            w, n = w_n.split(':')
            w = int(w)
            n = int(n)
            W[-1][c:c+n] += w
            c = c + n
    return W

In [14]:
W = _W_()

### ??

In [15]:
Dis = dirichlet(alpha = [.5,.5])

In [16]:
A = Dis.rvs(size = 5)
A

array([[0.99307776, 0.00692224],
       [0.97460825, 0.02539175],
       [0.64749364, 0.35250636],
       [0.74272889, 0.25727111],
       [0.89487849, 0.10512151]])

In [17]:
Dis.logpdf(x = A.T)

array([ 1.3452509 ,  0.70479552, -0.40606342, -0.31720537,  0.03712295])

In [18]:
Dis.logpdf(A[0])

1.3452509020688042

In [19]:
Dis.pdf(A.T)

array([3.83914967, 2.02343289, 0.66626791, 0.7281812 , 1.03782061])

## DLA

In [20]:
K = 10 #number of topics
alpha = 1/K* np.ones(K)
n_words = sum(lengths_of_documents)

In [21]:
#phi = [1/K*np.ones((l, K)) for l in lengths_of_documents] # a list of l x K matrix: z_nj ~ Mult(phi[n][j])

phi = 1/K * np.ones((n_words, K))
psi = 1/V * np.ones((K,V))# an K x V matrix whose rows are topics: beta_k ~ a Fir(psi[k])
gamma = np.ones((M, K)) # M Mixture parameters: theta_i ~ Dir(gamma[i])
alpha = 1/K * np.ones(K)
eta = 1

In [22]:
gamma.shape

(2246, 10)

In [42]:
def _phi_():
    temp = np.zeros(phi.shape)
    A = digamma(gamma) # M x K
    B = digamma(psi) # K x V
    B = B - B.sum(axis =1, keepdims = True)
    for m in range(M):
        temp[spd[m]:spd[m+1]] = (A[m] + B[:, W[m]].T).copy()
    return softmax(temp)

In [43]:
#_phi_()

In [44]:
def _gamma_():
    temp = np.zeros(gamma.shape)
    for m in range(M):
        temp[m] = alpha + phi[spd[m]:spd[m+1]].sum(axis = 0)
    return temp

In [45]:
_gamma_().shape

(2246, 10)

In [46]:
def _psi_():
    temp = np.zeros(psi.shape) # (K, V)
    for m in range(M):
        for i in range(len(W[m])):
            v = W[m][i]
            temp[:, v] += (phi[spd[m]:spd[m+1]][i]).copy()
    return eta + temp

In [47]:
for i in range(100):
    phi = _phi_()
    gamma = _gamma_()
    psi = _psi_()


In [48]:
I = np.argsort(psi)[-10:]
I

array([[    0,  6976,  6977, ...,  3495,  3477, 10472],
       [ 5236,  6976,  6977, ...,     2,     1,     0],
       [    0,  6976,  6977, ...,  3495,  3477, 10472],
       ...,
       [ 6009,  4071,  9100, ...,     2,     1,     0],
       [ 6009,  4071,  9100, ...,     2,     1,     0],
       [ 6009,  4071,  9100, ...,     2,     1,     0]])

In [49]:
np.array(Vocab)[I]

array([['i', 'classic', 'tibet', ..., 'wear', 'blocked', 'buffs'],
       ['vincennes', 'classic', 'tibet', ..., 'percent', 'new', 'i'],
       ['i', 'classic', 'tibet', ..., 'wear', 'blocked', 'buffs'],
       ...,
       ['steiger', 'confiscated', 'mead', ..., 'percent', 'new', 'i'],
       ['steiger', 'confiscated', 'mead', ..., 'percent', 'new', 'i'],
       ['steiger', 'confiscated', 'mead', ..., 'percent', 'new', 'i']],
      dtype='<U18')

### Optimizing $\alpha$ and $\eta$

In [50]:
def ell_alpha(alpha):
    A = np.log(Gamma(alpha.sum()))
    B = np.sum(np.log(Gamma(alpha+1e-7)))
    C = np.sum(digamma(gamma+1e-7) @ (alpha -1))
    return M*A+M*B+C

In [51]:
def d_ell_alpha(alpha):
    A = digamma(gamma) #(M,K)
    A = A - digamma(gamma.sum(axis = 0))
    return M * digamma(alpha.sum()) - M * digamma(alpha) + A.sum(axis =0)

In [52]:
ell_alpha(10*np.ones(K))

-202140504751.35583

In [53]:
d_ell_alpha(10*np.ones(K))

/var/folders/0y/hx4crw9178xfvr_01vxxcmkh0000gn/T/ipykernel_72282/985436813.py:3: RuntimeWarning: invalid value encountered in subtract
  A = A - digamma(gamma.sum(axis = 0))


array([             nan, -105770.93791114,  -56725.77348825,
        -40719.30191056,  -20925.22145423,  -13825.29072742,
        -12781.02619427,  -12573.02535615,  -12513.86763526,
        -12492.23443447])

In [54]:
def constraint1(alpha):
    return alpha
def constraint2(alpha):
    return alpha.sum() - 1
cons = [{'type':'ineq', 'fun': constraint1}]

In [55]:
Gamma(15)

87178291200.0

In [56]:
alpha = np.arange(K)
alpha = alpha/alpha.sum()

In [57]:
minimize(ell_alpha, alpha, jac = d_ell_alpha, method = 'SLSQP',  constraints = cons)

/var/folders/0y/hx4crw9178xfvr_01vxxcmkh0000gn/T/ipykernel_72282/985436813.py:3: RuntimeWarning: invalid value encountered in subtract
  A = A - digamma(gamma.sum(axis = 0))


     fun: 22460257419.790447
     jac: array([            nan, -10056.25848067, -11624.37339556, -12538.80846343,
        -1240.89225514,    733.99628344,  -1660.27064836,  -3926.32230514,
        -5737.79817369,  -7184.0044163 ])
 message: 'Inequality constraints incompatible'
    nfev: 1
     nit: 1
    njev: 1
  status: 4
 success: False
       x: array([0.        , 0.02222222, 0.04444444, 0.06666667, 0.08888889,
       0.11111111, 0.13333333, 0.15555556, 0.17777778, 0.2       ])

In [58]:
def ell_eta(eta):
    C = digamma(psi) # (K, V)
    D = digamma(psi.sum(axis =1, keepdims = True))
    return K * np.log(Gamma(V*eta)) - V*K*np.log(Gamma(eta)) + (eta-1) * (C-D).sum()

In [59]:
def d_ell_eta(eta):
    C = digamma(psi) # (K, V)
    D = digamma(psi.sum(axis =1, keepdims = True))
    return K*V * digamma(V*eta) - V*K*digamma(eta) + (C-D).sum()

In [60]:
def const1(eta):
    return eta
cons_eta = [{'type':'ineq', 'fun': const1}]

In [61]:
eta = 0.001
minimize(ell_eta, eta, jac = d_ell_eta, constraints = cons_eta)

     fun: 317689.59036443336
     jac: array([1.03989208e+08])
 message: 'Inequality constraints incompatible'
    nfev: 1
     nit: 1
    njev: 1
  status: 4
 success: False
       x: array([0.001])

In [ ]:
Gamma(3)